In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re
import datetime
import os.path

* 层次化索引
* 数据的联合及合并
* 重塑及转轴

## 一、层次化索引
> 层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。

In [2]:
data = pd.Series(np.random.randn(9),
   ...:                  index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
   ...:                         [1, 2, 3, 1, 3, 1, 2, 2, 3]])


In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

数据选取

In [4]:
data['a':'b']

a  1   -1.397390
   2   -1.641580
   3   -1.552073
b  1    1.785600
   3   -0.294151
dtype: float64

In [6]:
data.loc[:,[2,3]]

a  2   -1.641580
   3   -1.552073
b  3   -0.294151
c  2    0.089667
d  2   -0.779014
   3   -0.286791
dtype: float64

数据堆叠及展开

In [8]:
data.unstack().stack()

a  1   -1.397390
   2   -1.641580
   3   -1.552073
b  1    1.785600
   3   -0.294151
c  1    0.786713
   2    0.089667
d  2   -0.779014
   3   -0.286791
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
   ....:                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
   ....:                      columns=[['Ohio', 'Ohio', 'Colorado'],
   ....:                               ['Green', 'Red', 'Green']])

In [11]:
frame.loc['a',:]

Ohio     Colorado
  Green Red    Green
1     0   1        2
2     3   4        5

In [13]:
#创建层次索引以便复用
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 重排与分级排序

In [14]:
frame.swaplevel(0,1)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
2 a     3   4        5
1 b     6   7        8
2 b     9  10       11

In [16]:
frame.sort_index(level = 1)

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
b 1     6   7        8
a 2     3   4        5
b 2     9  10       11

### 根据级别汇总统计

In [17]:
frame.sum(level = 1)

Ohio     Colorado
  Green Red    Green
1     6   8       10
2    12  14       16

In [18]:
frame.sum(level = 1,axis = 1)

Green  Red
a 1      2    1
  2      8    4
b 1     14    7
  2     20   10

### 列索引

In [19]:
frame1 = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
   ....:                       'c': ['one', 'one', 'one', 'two', 'two',
   ....:                             'two', 'two'],
   ....:                       'd': [0, 1, 2, 0, 1, 2, 3]})


In [20]:
frame2 = frame1.set_index('c')

In [21]:
frame2

,a,b,d
c,,,
one,0,7,0
one,1,6,1
one,2,5,2
two,3,4,0
two,4,3,1
two,5,2,2
two,6,1,3


In [23]:
frame2.reset_index().reindex(['a','b','c','d'],axis =1)

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


> 合并，索引将会被丢弃；
> 重复列名将会被重命名

In [3]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
   ....:                      'key2': ['one', 'two', 'one'],
   ....:                      'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
   ....:                       'key2': ['one', 'one', 'one', 'two'],
   ....:                       'rval': [4, 5, 6, 7]})


In [6]:
frame8 = pd.merge(left,right,on = 'key1',suffixes = ('_left','_right'))

In [7]:
frame8

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引上的合并
> left_index,right_index

#### DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并

In [8]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
   ....:                      index=['a', 'c', 'e'],
   ....:                      columns=['Ohio', 'Nevada'])

In [69]: right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
   ....:                       index=['b', 'c', 'd', 'e'],
   ....:                       columns=['Missouri', 'Alabama'])


In [9]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


### 轴向连接
>另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。NumPy的concatenation函数可以用NumPy数组来做：

In [10]:
arr = np.arange(12).reshape((3, 4))


In [13]:
np.concatenate((arr,arr))

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [14]:
In [82]: s1 = pd.Series([0, 1], index=['a', 'b'])

In [83]: s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [84]: s3 = pd.Series([5, 6], index=['f', 'g'])


In [16]:
pd.concat([s1,s2,s3],axis = 1,join = 'inner')

,0,1,2
